In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from time import sleep
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st

In [24]:
conexion = create_engine("postgresql://{user}:{pw}@{host}:{port}/{db}"
                         .format(user="postgres",
                                 pw="postgres",
                                 host="database-grupo5.cgmzd7suyc4v.us-east-1.rds.amazonaws.com",
                                 port="5432",
                                 db="postgres",
                                 echo=False))
if conexion.connect():
  print("conexion exitosa")

conexion exitosa


In [96]:
g_n=pd.read_csv('https://raw.githubusercontent.com/MAKACT/proyectoConsumoEnergiaEquipo5/main/Consumo%20Gas%20Natural%20Exajoules.csv')
g_n=g_n.drop(columns={'2021.1','2011-21','2021.2'})
g_n.fillna(0, inplace=True)
g_n.replace("-", 0, inplace=True)
g_n.replace("^", 0, inplace=True)
g_n.replace("♦", 0, inplace=True)
indexN=g_n[g_n['Pais']=='Other Europe (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other CIS (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Middle East (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Western Africa (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Northern Africa (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Asia Pacific (BP)'].index
g_n.drop(indexN, inplace=True)

g_n

,Pais,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Canada,0.78,0.85,0.91,1.01,1.13,1.25,1.33,1.50,1.59,...,3.58,3.80,3.95,3.97,3.78,3.96,4.16,4.22,4.08,4.29
1,Mexico,0.29,0.31,0.26,0.28,0.35,0.36,0.37,0.40,0.44,...,2.65,2.80,2.84,2.91,2.99,3.10,3.15,3.17,3.01,3.18
2,US,14.97,16.14,17.04,18.24,19.63,20.70,21.34,21.55,21.38,...,24.77,25.45,26.00,26.77,26.97,26.64,29.58,30.62,29.95,29.76
3,Argentina,0.15,0.16,0.17,0.19,0.19,0.21,0.23,0.26,0.29,...,1.64,1.66,1.66,1.68,1.74,1.74,1.75,1.68,1.58,1.65
4,Brazil,0,0,0,0,0,0,0,0.01,0.01,...,1.17,1.38,1.46,1.55,1.34,1.35,1.29,1.29,1.13,1.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Sri Lanka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,Taiwan,0.01,0.01,0.01,0.02,0.02,0.02,0.04,0.05,0.05,...,0.64,0.65,0.68,0.73,0.75,0.84,0.85,0.84,0.90,0.98
89,Thailand,0,0,0,0,0,0,0,0,0,...,1.75,1.76,1.80,1.83,1.82,1.80,1.80,1.83,1.69,1.69
90,Vietnam,0,0,0,0,0,0,0,0,0,...,0.32,0.34,0.35,0.37,0.37,0.34,0.35,0.35,0.32,0.26


In [97]:
g_n.Pais.unique()

array(['Canada', 'Mexico', 'US', 'Argentina', 'Brazil', 'Chile',
       'Colombia', 'Ecuador', 'Peru', 'Trinidad & Tobago', 'Venezuela',
       'Central America', 'Other Caribbean', 'Other South America',
       'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands',
       'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey',
       'Ukraine', 'United Kingdom', 'Other Europe', 'Azerbaijan',
       'Belarus', 'Kazakhstan', 'Russian Federation', 'Turkmenistan',
       'USSR', 'Uzbekistan', 'Other CIS', 'Iran', 'Iraq', 'Israel',
       'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates',
       'Other Middle East', 'Algeria', 'Egypt', 'Morocco', 'South Africa',
       'Eastern Africa', 'Middle Africa

In [24]:
#query=('SELECT *FROM "Emisiones_CO2";')
emisiones = pd.read_sql_query('SELECT *FROM "Emisiones_CO2";', con=conexion)
emisiones

,ID_Emisiones_CO2,Id_Pais,Id_anio,Emisiones_CO2
0,1,44,26,480.4
1,2,160,26,315.9
2,3,297,26,5275.4
3,4,11,26,123.9
4,5,35,26,217.1
...,...,...,...,...
2619,2620,262,57,628.7
2620,2621,273,57,285.3
2621,2622,276,57,301.8
2622,2623,304,57,339.8


In [16]:
#query=('SELECT *FROM "Paises";')
paises = pd.read_sql_query('SELECT *FROM "Paises";', con=conexion)
paises

,Id_Pais,Codigo_pais,Pais
0,1,AFG,Afghanistan
1,2,0,Africa
2,3,0,Africa (BP)
3,4,0,Africa (Shift)
4,5,ALB,Albania
...,...,...,...
309,310,0,World
310,311,YEM,Yemen
311,312,0,Yugoslavia
312,313,ZMB,Zambia


In [17]:
query=('SELECT *FROM "Años";')
años = pd.read_sql_query('SELECT *FROM "Años";', con=conexion)
años

,Id_anio,Anio
0,1,1965
1,2,1966
2,3,1967
3,4,1968
4,5,1969
5,6,1970
6,7,1971
7,8,1972
8,9,1973
9,10,1974


In [73]:
emisiones80=pd.merge(emisiones,paises,on='Id_Pais')
emisiones80=pd.merge(emisiones80,años,on='Id_anio')
emisiones80.drop(columns={'Id_Pais','Id_anio'}, inplace=True)
emisiones80

NameError: name 'emisiones' is not defined

In [15]:
emisiones80.dtypes

ID_Emisiones_CO2      int64
Emisiones_CO2       float64
Codigo_pais          object
Pais                 object
Anio                  int64
dtype: object

In [28]:
emisiones_año=emisiones80.groupby(by=['Anio']).mean()
emisiones_año



,ID_Emisiones_CO2,Emisiones_CO2
Anio,,
1990,38.676923,349.035385
1991,120.676923,349.596923
1992,202.676923,350.026154
1993,284.676923,350.680000
1994,366.676923,354.147692
1995,448.676923,359.796923
1996,530.676923,369.423077
1997,612.676923,371.893846
1998,694.676923,372.772308


In [71]:
emisiones_año['Variacion']=(emisiones_año['Emisiones_CO2'].pct_change())*100
emisiones_año

,ID_Emisiones_CO2,Emisiones_CO2,Variacion
Anio,,,
1990,38.676923,349.035385,NaN
1991,120.676923,349.596923,0.160883
1992,202.676923,350.026154,0.122779
1993,284.676923,350.680000,0.186799
1994,366.676923,354.147692,0.988848
1995,448.676923,359.796923,1.595162
1996,530.676923,369.423077,2.675441
1997,612.676923,371.893846,0.668818
1998,694.676923,372.772308,0.236213


In [85]:
emisionesk = pd.read_sql_query('SELECT *FROM "Emisiones_CO2";', con=conexion)
paisesk = pd.read_sql_query('SELECT *FROM "Paises";', con=conexion)
añosk = pd.read_sql_query('SELECT *FROM "Años";', con=conexion)
emisiones80k=pd.merge(emisionesk,paisesk,on='Id_Pais')
emisiones80k=pd.merge(emisiones80k,añosk,on='Id_anio')
emisiones80k.drop(columns={'Id_Pais','Id_anio'}, inplace=True)
emisiones_añok=emisiones80k.groupby(by=['Anio']).mean()
emisiones_añok['Variacion']=(emisiones_añok['Emisiones_CO2'].pct_change())*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=emisiones_añok.index, y=emisiones_añok['Variacion'],
                    mode='lines'
                     ))
fig.update_layout(title='Porcentaje de incremento de Emisiones CO2 en el mundo por año de  1990 a 2021',
                   xaxis_title='Año',
                   yaxis_title='Porcentaje de incremento')
#st.plotly(fig, use_container_width=True)
#fig.show()

In [42]:
emisiones80[emisiones80['Pais'].str.contains('Other')==True]
emisiones80['Anio']=emisiones80['Anio'].astype(str)
emisiones_pais=emisiones80.groupby(by=['Pais']).mean()

emisiones_pais

,ID_Emisiones_CO2,Emisiones_CO2
Pais,,
Argentina,1275.0,177.353125
Australia,1338.0,407.290625
Austria,1286.0,66.078125
Azerbaijan,1313.0,48.337500
Belarus,1314.0,64.087500
...,...,...
Ukraine,1310.0,352.534375
United Kingdom,1311.0,534.896875
United States,1274.0,5675.496875


In [49]:
emisiones_pais.drop(['Other Asia Pacific (BP)',
       'Other CIS (BP)', 'Other Caribbean (BP)', 'Other Europe (BP)',
       'Other Middle East (BP)', 'Other Northern Africa (BP)',
       'Other South America (BP)', 'Other South and Central America (BP)',
       'Other Western Africa (BP)'], axis=0, inplace= True)
emisiones_pais

KeyError: "['Other Asia Pacific (BP)', 'Other CIS (BP)', 'Other Caribbean (BP)', 'Other Europe (BP)', 'Other Middle East (BP)', 'Other Northern Africa (BP)', 'Other South America (BP)', 'Other South and Central America (BP)', 'Other Western Africa (BP)'] not found in axis"

In [52]:
emisiones_pais=emisiones_pais.sort_values('Emisiones_CO2',ascending=True)
emisiones_pais

,ID_Emisiones_CO2,Emisiones_CO2
Pais,,
North Macedonia,1301.0,11.750000
Lithuania,1299.0,15.046875
Croatia,1289.0,19.950000
Estonia,1292.0,22.650000
New Zealand,1346.0,35.318750
Ecuador,1279.0,36.468750
Norway,1302.0,37.068750
Peru,1280.0,39.712500
Ireland,1297.0,41.221875


In [56]:
len(emisiones_pais)

56

In [66]:
from turtle import title


fig = go.Figure()
fig.add_trace(go.Bar(x=emisiones_pais.index, y=emisiones_pais['Emisiones_CO2']))
                    #mode='lines',
                    #"title='Producción Carbón'))
fig.update_layout(title='Promedio de Emisiones CO2 por país de 1980 a 2021',
                  xaxis_tickfont_size=14,
                 yaxis=dict(
                        title='Emisones(Millones de Toneladas)',
                        titlefont_size=12,
                        tickfont_size=12,
    )
                 )
                  # xaxis_title='Pais',
                  # yaxis_title='Emisones(Millones de Toneladas)')

fig.show()

In [100]:
irradianzak = pd.read_sql_query('SELECT *FROM "Irradianza";', con=conexion)
irradianzak.rename(columns={'Id_Anio':'Id_anio'}, inplace=True)
irradianzak=pd.merge(irradianzak,paisesk,on='Id_Pais')
irradianzak=pd.merge(irradianzak,añosk,on='Id_anio')
irradianzak.drop(columns={'Id_Pais','Id_anio'}, inplace=True)
irradianzak.sort_values('Irradianza',ascending=True)

,Id_Irradianza,Irradianza,Codigo_pais,Pais,Anio
121,121,330722.583333,FIN,Finland,2012
274,274,332896.750000,FIN,Finland,2015
325,325,333164.000000,FIN,Finland,2016
89,89,333900.666667,NOR,Norway,2011
376,376,338766.000000,FIN,Finland,2017
...,...,...,...,...,...
119,119,951469.416667,EGY,Egypt,2012
374,374,953696.000000,EGY,Egypt,2017
68,68,954180.416667,EGY,Egypt,2011
17,17,957870.333333,EGY,Egypt,2010


In [103]:
irra20=irradianzak[irradianzak['Anio']==2020]
irra20.sort_values('Irradianza',ascending=True)

,Id_Irradianza,Irradianza,Codigo_pais,Pais,Anio
548,548,364215.000000,NOR,Norway,2020
529,529,370052.500000,FIN,Finland,2020
560,560,393738.000000,SWE,Sweden,2020
536,536,410909.583333,IRL,Ireland,2020
541,541,417900.083333,LVA,Latvia,2020
528,528,419725.750000,EST,Estonia,2020
556,556,419861.583333,RUS,Russia,2020
542,542,429099.250000,LTU,Lithuania,2020
514,514,430867.833333,BLR,Belarus,2020
518,518,446920.583333,CAN,Canada,2020


In [105]:
queryFranco='SELECT a."Anio", pa."Pais", pa."Codigo_pais", (er."Consumo_renovables"+en."Consumo_Nuclear")/et."Consumo_total" as "Energia_Limpia" FROM "Energia_renovables" er JOIN "Energia_nuclear_con_ceros" en ON  er."Id_Pais" = en."Id_Pais" AND er."Id_anio" = en."Id_anio" JOIN "Energia_total" et ON er."Id_Pais" = et."Id_Pais" AND er."Id_anio" = et."Id_anio" JOIN "Paises" pa ON er."Id_Pais" = pa."Id_Pais" JOIN "Años" a ON er."Id_anio"=a."Id_anio";'
FR_Energias_limpias = pd.read_sql_query(queryFranco, con=conexion)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other CIS (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Northern Africa (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Western Africa (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Asia Pacific (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other South America (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Eastern Africa (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Caribbean (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Europe (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other Middle East (BP)"].index, inplace=True)
FR_Energias_limpias.drop(FR_Energias_limpias[FR_Energias_limpias.Pais == "Other South and Central America (BP)"].index, inplace=True)

In [106]:
FR_Energias_limpias

,Anio,Pais,Codigo_pais,Energia_Limpia
0,1995,Argentina,ARG,0.031556
1,1995,Australia,AUS,0.002355
2,1995,Austria,AUT,0.016527
3,1995,Azerbaijan,AZE,0.000000
4,1995,Belarus,BLR,0.000000
...,...,...,...,...
1939,2021,Ukraine,UKR,0.266153
1940,2021,United Kingdom,GBR,0.229993
1941,2021,United States,USA,0.160044
1942,2021,Venezuela,VEN,0.000463


In [74]:
carbon=pd.read_sql_query('SELECT *FROM "Energia_Carbon";', con=conexion)
carbon.rename(columns={'Id_Anio':'Id_anio'},inplace=True)
carbon

,Id_Energia_Carbon,Id_Pais,Id_anio,Consumo_Carbon,Produccion_Carbono
0,1,44,17,0.95,0.97
1,2,160,17,0.09,0.07
2,3,297,17,15.94,18.42
3,4,35,17,0.24,0.12
4,5,54,17,0.10,0.12
...,...,...,...,...,...
1430,1431,225,57,0.67,0.14
1431,1432,262,57,3.04,0.02
1432,1433,276,57,0.81,0.15
1433,1434,304,57,2.15,1.12


In [75]:
petroleo=pd.read_sql_query('SELECT *FROM "Energia_petroleo";', con=conexion)

petroleo

,ID_Energia_petroleo,Id_Pais,Id_anio,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo
0,1,44,1,2.474186,2.054378,0.0
1,2,160,1,0.705634,0.808353,0.0
2,3,297,1,25.706527,20.128443,0.0
3,4,11,1,1.018257,0.616314,0.0
4,5,35,1,0.683304,0.214370,0.0
...,...,...,...,...,...,...
2275,2276,124,57,3.284773,1.545250,0.0
2276,2277,0,57,1.703794,1.279520,0.0
2277,2278,276,57,2.710886,0.888742,0.0
2278,2279,304,57,1.036121,0.428740,0.0


In [76]:
gas=pd.read_sql_query('SELECT *FROM "Gas_Natural";', con=conexion)
gas

,ID_Gas_Natural,Id_Pais,Id_anio,Reserva_Gas,Produccion_gas,Consumo_gas
0,1,44,16,2.4,2.4,1.79
1,2,160,16,1.8,1.8,0.80
2,3,297,16,5.4,5.4,19.21
3,4,11,16,0.6,0.6,0.40
4,5,35,16,0.1,0.1,0.04
...,...,...,...,...,...,...
1922,1923,0,56,0.9,0.9,1.38
1923,1924,225,56,0.4,0.4,1.48
1924,1925,276,56,0.1,0.1,1.69
1925,1926,304,56,0.6,0.6,0.32


In [77]:
ene_fosil=pd.merge(carbon,petroleo, on='Id_anio')
ene_fosil=pd.merge(ene_fosil,gas, on='Id_anio')
ene_fosil.drop(columns={'Id_Pais_x','ID_Energia_petroleo','Id_Pais_y','ID_Gas_Natural'},inplace=True)
ene_fosil

,Id_Energia_Carbon,Id_anio,Consumo_Carbon,Produccion_Carbono,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo,Id_Pais,Reserva_Gas,Produccion_gas,Consumo_gas
0,1,17,0.95,0.97,4.189146,3.595162,245.938101,44,2.5,2.5,1.74
1,1,17,0.95,0.97,4.189146,3.595162,245.938101,160,2.1,2.1,0.87
2,1,17,0.95,0.97,4.189146,3.595162,245.938101,297,5.5,5.5,18.75
3,1,17,0.95,0.97,4.189146,3.595162,245.938101,11,0.6,0.6,0.42
4,1,17,0.95,0.97,4.189146,3.595162,245.938101,35,0.1,0.1,0.03
...,...,...,...,...,...,...,...,...,...,...,...
2631995,1400,56,1.17,1.11,1.170103,0.464468,7.953222,0,0.9,0.9,1.38
2631996,1400,56,1.17,1.11,1.170103,0.464468,7.953222,225,0.4,0.4,1.48
2631997,1400,56,1.17,1.11,1.170103,0.464468,7.953222,276,0.1,0.1,1.69
2631998,1400,56,1.17,1.11,1.170103,0.464468,7.953222,304,0.6,0.6,0.32


In [78]:
ene_fosil_consumo=ene_fosil.loc[:,('Id_anio','Id_Pais','Consumo_Carbon','Consumo_Petroleo','Consumo_gas')]
ene_fosil_consumo['Total_Consumo']=ene_fosil_consumo['Consumo_Carbon']+ene_fosil_consumo['Consumo_Petroleo']+ene_fosil_consumo['Consumo_gas']
ene_fosil_consumo

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo
0,17,44,0.95,4.189146,1.74,6.879146
1,17,160,0.95,4.189146,0.87,6.009146
2,17,297,0.95,4.189146,18.75,23.889146
3,17,11,0.95,4.189146,0.42,5.559146
4,17,35,0.95,4.189146,0.03,5.169146
...,...,...,...,...,...,...
2631995,56,0,1.17,1.170103,1.38,3.720103
2631996,56,225,1.17,1.170103,1.48,3.820103
2631997,56,276,1.17,1.170103,1.69,4.030103
2631998,56,304,1.17,1.170103,0.32,2.660103


In [79]:
ene_fosil_consumo_total=pd.merge(ene_fosil_consumo,paises, on='Id_Pais')
ene_fosil_consumo_total=pd.merge(ene_fosil_consumo_total,años, on='Id_anio')

ene_fosil_consumo_total

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo,Codigo_pais,Pais,Anio
0,17,44,0.95,4.189146,1.74,6.879146,CAN,Canada,1981
1,17,44,0.95,2.670692,1.74,5.360692,CAN,Canada,1981
2,17,44,0.95,35.857836,1.74,38.547836,CAN,Canada,1981
3,17,44,0.95,1.062918,1.74,3.752918,CAN,Canada,1981
4,17,44,0.95,2.324574,1.74,5.014574,CAN,Canada,1981
...,...,...,...,...,...,...,...,...,...
1903995,56,210,1.17,3.121762,0.42,4.711762,0,Other Asia Pacific (BP),2020
1903996,56,210,1.17,1.672532,0.42,3.262532,0,Other Asia Pacific (BP),2020
1903997,56,210,1.17,2.710886,0.42,4.300886,0,Other Asia Pacific (BP),2020
1903998,56,210,1.17,1.100879,0.42,2.690879,0,Other Asia Pacific (BP),2020


In [80]:
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Europe (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other CIS (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Middle East (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Western Africa (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Northern Africa (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Asia Pacific (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
#ene_fosil_consumo_total['Id_Pais']=ene_fosil_consumo_total['Id_Pais'].astype()
ene_fosil_consumo_total

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo,Codigo_pais,Pais,Anio
0,17,44,0.95,4.189146,1.74,6.879146,CAN,Canada,1981
1,17,44,0.95,2.670692,1.74,5.360692,CAN,Canada,1981
2,17,44,0.95,35.857836,1.74,38.547836,CAN,Canada,1981
3,17,44,0.95,1.062918,1.74,3.752918,CAN,Canada,1981
4,17,44,0.95,2.324574,1.74,5.014574,CAN,Canada,1981
...,...,...,...,...,...,...,...,...,...
1902595,56,304,1.17,3.121762,0.32,4.611762,VNM,Vietnam,2020
1902596,56,304,1.17,1.672532,0.32,3.162532,VNM,Vietnam,2020
1902597,56,304,1.17,2.710886,0.32,4.200886,VNM,Vietnam,2020
1902598,56,304,1.17,1.100879,0.32,2.590879,VNM,Vietnam,2020


In [81]:
ene_fosil_consumo_total[(ene_fosil_consumo_total.Pais=='Canada') & (ene_fosil_consumo_total.Anio ==1981)]


,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo,Codigo_pais,Pais,Anio
0,17,44,0.95,4.189146,1.74,6.879146,CAN,Canada,1981
1,17,44,0.95,2.670692,1.74,5.360692,CAN,Canada,1981
2,17,44,0.95,35.857836,1.74,38.547836,CAN,Canada,1981
3,17,44,0.95,1.062918,1.74,3.752918,CAN,Canada,1981
4,17,44,0.95,2.324574,1.74,5.014574,CAN,Canada,1981
...,...,...,...,...,...,...,...,...,...
1395,17,44,1.21,0.966898,1.74,3.916898,CAN,Canada,1981
1396,17,44,1.21,0.390779,1.74,3.340779,CAN,Canada,1981
1397,17,44,1.21,0.489031,1.74,3.439031,CAN,Canada,1981
1398,17,44,1.21,0.078156,1.74,3.028156,CAN,Canada,1981


In [82]:
año90=ene_fosil_consumo_total[ene_fosil_consumo_total.Anio>=1990]
canada=año90[(año90.Pais=='Canada')].groupby(by=['Anio']).mean()


canada

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo
Anio,,,,,,
1990,26.0,44.0,2.501429,2.405912,2.30,7.207340
1991,27.0,44.0,2.475143,2.396477,2.31,7.181620
1992,28.0,44.0,2.476000,2.443650,2.45,7.369650
1993,29.0,44.0,2.491714,2.445492,2.60,7.537206
1994,30.0,44.0,2.500857,2.494283,2.69,7.685141
1995,31.0,44.0,2.517143,2.517898,2.78,7.815040
1996,32.0,44.0,2.573714,2.576235,2.83,7.979950
1997,33.0,44.0,2.571143,2.664272,2.88,8.115415
1998,34.0,44.0,2.550857,2.691403,2.84,8.082260


In [83]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=canada.index,
    y=canada['Consumo_Carbon'],
    name='Carbon',
    marker_color= 'lightslategray'
))
fig.add_trace(go.Bar(
    x=canada.index,
    y=canada['Consumo_Petroleo'],
    name='Petroleo',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=canada.index,
    y=canada['Consumo_gas'],
    name='Gas Natural',
    marker_color='lightsalmon'
))
fig.show()

In [84]:
ene_fosil_consumo_total_año=ene_fosil_consumo_total.groupby(by=['Anio']).mean()
ene_fosil_consumo_total_año

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo
Anio,,,,,,
1981,17.0,159.7,2.062571,2.256858,1.062667,5.382096
1982,18.0,159.7,2.079429,2.215714,1.024667,5.319809
1983,19.0,159.7,2.147429,2.212030,1.004333,5.363792
1984,20.0,159.7,2.236000,2.280137,1.071333,5.587470
1985,21.0,159.7,2.320286,2.174850,1.623000,6.118136
1986,22.0,159.7,2.351714,2.235700,1.614333,6.201747
1987,23.0,159.7,2.445143,2.292865,1.697333,6.435341
1988,24.0,159.7,2.520571,2.362033,1.769667,6.652271
1989,25.0,159.7,2.542000,2.407642,1.839333,6.788976


In [85]:
ene_fosil_consumo_total_pais=ene_fosil_consumo_total.groupby(by=['Pais']).mean()
ene_fosil_consumo_total_pais

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo,Anio
Pais,,,,,,,
Argentina,36.5,11.0,3.169821,2.918628,1.14650,7.23495,2000.5
Australia,36.5,18.0,3.169821,2.918628,0.86625,6.95470,2000.5
Azerbaijan,36.5,21.0,3.169821,2.918628,0.32125,6.40970,2000.5
Brazil,36.5,35.0,3.169821,2.918628,0.57650,6.66495,2000.5
Canada,36.5,44.0,3.169821,2.918628,2.97400,9.06245,2000.5
China,36.5,53.0,3.169821,2.918628,2.82025,8.90870,2000.5
Colombia,36.5,54.0,3.169821,2.918628,0.23900,6.32745,2000.5
Denmark,36.5,66.0,3.169821,2.918628,0.11975,6.20820,2000.5
Egypt,36.5,74.0,3.169821,2.918628,0.92950,7.01795,2000.5


In [86]:
len(ene_fosil_consumo_total_pais.index)

30

In [250]:
eoil=pd.read_sql_query('SELECT *FROM "Energia_Eolica";', con=conexion)
eoil

,Id_Energia_Eolica,Id_Pais,Id_anio,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica
0,1,66,21,0,0.1,0
1,2,66,22,0,0.1,0
2,3,66,23,0,0.2,0
3,4,66,24,0,0.3,0
4,5,297,25,0.02,2.1,0
...,...,...,...,...,...,...
1550,1551,300,57,0,0,1514
1551,1552,222,57,0,0,2258
1552,1553,132,57,0,0,622
1553,1554,283,57,0,0,245


In [251]:
solar=pd.read_sql_query('SELECT *FROM "Energia_solar";', con=conexion)
solar


,Id_Energia_solar,Id_Pais,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar
0,1,297,25,0.00,0.3,0
1,2,297,26,0.00,0.4,0
2,3,297,27,0.01,0.5,0
3,4,297,28,0.00,0.4,0
4,5,297,29,0.01,0.5,0
...,...,...,...,...,...,...
1238,1239,0,57,0.00,0.0,514
1239,1240,222,57,0.00,0.0,3816
1240,1241,132,57,0.00,0.0,1521
1241,1242,209,57,0.00,0.0,2268


In [252]:
hidro=pd.read_sql_query('SELECT *FROM "Energia_Hidroelectrica";', con=conexion)
hidro.rename(columns={'Id_Anio':'Id_anio'}, inplace=True)
hidro


,Id_Energia_Hidroelectrica,Id_Pais,Id_anio,Produccion_Hidroelectrica,Consumo_Hidroelectrica
0,0,44,1.0,0.425157,1.26
1,1,160,1.0,0.032040,0.09
2,2,297,1.0,0.716394,2.12
3,3,11,1.0,0.004320,0.01
4,4,35,1.0,0.091799,0.27
...,...,...,...,...,...
4099,4099,267,57.0,0.025560,0.07
4100,4100,273,57.0,0.012600,0.03
4101,4101,276,57.0,0.016200,0.04
4102,4102,304,57.0,0.273238,0.71


In [253]:
geo=pd.read_sql_query('SELECT *FROM "Energia_Geotermica";', con=conexion)
geo.rename(columns={'Id_Anio':'Id_anio'}, inplace=True)
geo


,Id_Energia_Geotermica,Id_Pais,Id_anio,Produccion_Geotermica,Consumo_Geotermica
0,0,44,1.0,0.00000,0
1,1,160,1.0,0.00000,0
2,2,297,1.0,0.04788,0.15
3,3,11,1.0,0.00000,0
4,4,35,1.0,0.00000,0
...,...,...,...,...,...
4042,4042,267,57.0,0.00036,0
4043,4043,273,57.0,0.00720,0.02
4044,4044,276,57.0,0.04680,0.15
4045,4045,304,57.0,0.00072,0


In [254]:
nuclear=pd.read_sql_query('SELECT *FROM "Energia_nuclear_con_ceros";', con=conexion)
nuclear

,ID_Energia_nuclear,Id_Pais,Id_anio,Consumo_Nuclear,Produccion_Nuclear
0,1,44,31,0.99,97.2
1,2,160,31,0.09,8.4
2,3,297,31,7.24,708.8
3,4,11,31,0.07,7.1
4,5,35,31,0.03,2.5
...,...,...,...,...,...
2479,2480,267,57,0.00,0.0
2480,2481,273,57,0.25,27.8
2481,2482,276,57,0.00,0.0
2482,2483,304,57,0.00,0.0


In [188]:
todas=pd.merge(solar,eoil, on='Id_anio')
todas=pd.merge(todas,hidro, on='Id_anio')
todas.drop(columns={'Id_Pais_x','Id_Pais_y','Id_Energia_Eolica','Id_Energia_Hidroelectrica'},inplace=True)
todas=pd.merge(todas, años, on='Id_anio')
#todas.drop(columns={'Id_anio'},inplace=True)
todas80=todas[todas['Anio']>=1980]

In [189]:
todas80=pd.merge(todas80,paises, on='Id_Pais')
todas80


,Id_Energia_solar,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Id_Pais,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,1,25,0.0,0.3,0,0.02,2.1,0,44,1.045432,3.09,1989,CAN,Canada
1,1,25,0.0,0.3,0,0,0.4,0,44,1.045432,3.09,1989,CAN,Canada
2,2,26,0.0,0.4,0,0.03,2.8,0,44,1.064511,3.15,1990,CAN,Canada
3,2,26,0.0,0.4,0,0.01,0.6,0,44,1.064511,3.15,1990,CAN,Canada
4,2,26,0.0,0.4,0,0,0.1,0,44,1.064511,3.15,1990,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797579,1243,57,0.0,0.0,16660,0,0,1514,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797580,1243,57,0.0,0.0,16660,0,0,2258,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797581,1243,57,0.0,0.0,16660,0,0,622,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797582,1243,57,0.0,0.0,16660,0,0,245,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)


In [171]:
todas80.drop(columns={'Id_Pais','Codigo_pais'},inplace=True)
"""todas80.drop(['Other Asia Pacific (BP)',
       'Other CIS (BP)', 'Other Caribbean (BP)', 'Other Europe (BP)',
       'Other Middle East (BP)', 'Other Northern Africa (BP)',
       'Other South America (BP)', 'Other South and Central America (BP)',
       'Other Western Africa (BP)'], axis=0, inplace= True)"""
todas80

KeyError: "['Codigo_pais', 'Id_Pais'] not found in axis"

In [190]:
todas80

,Id_Energia_solar,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Id_Pais,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,1,25,0.0,0.3,0,0.02,2.1,0,44,1.045432,3.09,1989,CAN,Canada
1,1,25,0.0,0.3,0,0,0.4,0,44,1.045432,3.09,1989,CAN,Canada
2,2,26,0.0,0.4,0,0.03,2.8,0,44,1.064511,3.15,1990,CAN,Canada
3,2,26,0.0,0.4,0,0.01,0.6,0,44,1.064511,3.15,1990,CAN,Canada
4,2,26,0.0,0.4,0,0,0.1,0,44,1.064511,3.15,1990,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797579,1243,57,0.0,0.0,16660,0,0,1514,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797580,1243,57,0.0,0.0,16660,0,0,2258,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797581,1243,57,0.0,0.0,16660,0,0,622,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)
5797582,1243,57,0.0,0.0,16660,0,0,245,210,0.268558,0.70,2021,0,Other Asia Pacific (BP)


In [191]:
indexN=todas80[todas80['Pais']=='Other Asia Pacific (BP)'].index
todas80.drop(indexN, inplace=True)
todas80

,Id_Energia_solar,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Id_Pais,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,1,25,0.0,0.3,0,0.02,2.1,0,44,1.045432,3.09,1989,CAN,Canada
1,1,25,0.0,0.3,0,0,0.4,0,44,1.045432,3.09,1989,CAN,Canada
2,2,26,0.0,0.4,0,0.03,2.8,0,44,1.064511,3.15,1990,CAN,Canada
3,2,26,0.0,0.4,0,0.01,0.6,0,44,1.064511,3.15,1990,CAN,Canada
4,2,26,0.0,0.4,0,0,0.1,0,44,1.064511,3.15,1990,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5717057,1243,57,0.0,0.0,16660,0,0,1514,304,0.273238,0.71,2021,VNM,Vietnam
5717058,1243,57,0.0,0.0,16660,0,0,2258,304,0.273238,0.71,2021,VNM,Vietnam
5717059,1243,57,0.0,0.0,16660,0,0,622,304,0.273238,0.71,2021,VNM,Vietnam
5717060,1243,57,0.0,0.0,16660,0,0,245,304,0.273238,0.71,2021,VNM,Vietnam


In [180]:
todas80['Pais'].unique()

array(['Canada', 'Mexico', 'United States', 'Argentina', 'Brazil',
       'Chile', 'Colombia', 'Ecuador', 'Peru', 'Venezuela',
       'Other South and Central America (BP)', 'Other Caribbean (BP)',
       'Other South America (BP)', 'Austria', 'Belgium', 'Bulgaria',
       'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland',
       'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands',
       'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom', 'Other Europe (BP)', 'Azerbaijan', 'Belarus',
       'Kazakhstan', 'Russia', 'Other CIS (BP)', 'Iran', 'Oman',
       'Other Middle East (BP)', 'Egypt', 'Morocco', 'South Africa',
       'Eastern Africa (BP)', 'Other Western Africa (BP)',
       'Other Northern Africa (BP)', 'Australia', 'China', 'India',
       'Indonesia', 'Japan', 'New Zealand', 'Pakistan', 'Philippines',

In [192]:
indexN=todas80[todas80['Pais']=='Other Europe (BP)'].index
todas80.drop(indexN, inplace=True)
todas80

,Id_Energia_solar,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Id_Pais,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,1,25,0.0,0.3,0,0.02,2.1,0,44,1.045432,3.09,1989,CAN,Canada
1,1,25,0.0,0.3,0,0,0.4,0,44,1.045432,3.09,1989,CAN,Canada
2,2,26,0.0,0.4,0,0.03,2.8,0,44,1.064511,3.15,1990,CAN,Canada
3,2,26,0.0,0.4,0,0.01,0.6,0,44,1.064511,3.15,1990,CAN,Canada
4,2,26,0.0,0.4,0,0,0.1,0,44,1.064511,3.15,1990,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5717057,1243,57,0.0,0.0,16660,0,0,1514,304,0.273238,0.71,2021,VNM,Vietnam
5717058,1243,57,0.0,0.0,16660,0,0,2258,304,0.273238,0.71,2021,VNM,Vietnam
5717059,1243,57,0.0,0.0,16660,0,0,622,304,0.273238,0.71,2021,VNM,Vietnam
5717060,1243,57,0.0,0.0,16660,0,0,245,304,0.273238,0.71,2021,VNM,Vietnam


In [193]:
indexN=todas80[todas80['Pais']=='Other CIS (BP)'].index
todas80.drop(indexN, inplace=True)
indexN=todas80[todas80['Pais']=='Other Middle East (BP)'].index
todas80.drop(indexN, inplace=True)
indexN=todas80[todas80['Pais']=='Other Western Africa (BP)'].index
todas80.drop(indexN, inplace=True)
indexN=todas80[todas80['Pais']=='Other Northern Africa (BP)'].index
todas80.drop(indexN, inplace=True)

todas80

,Id_Energia_solar,Id_anio,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Id_Pais,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,1,25,0.0,0.3,0,0.02,2.1,0,44,1.045432,3.09,1989,CAN,Canada
1,1,25,0.0,0.3,0,0,0.4,0,44,1.045432,3.09,1989,CAN,Canada
2,2,26,0.0,0.4,0,0.03,2.8,0,44,1.064511,3.15,1990,CAN,Canada
3,2,26,0.0,0.4,0,0.01,0.6,0,44,1.064511,3.15,1990,CAN,Canada
4,2,26,0.0,0.4,0,0,0.1,0,44,1.064511,3.15,1990,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5717057,1243,57,0.0,0.0,16660,0,0,1514,304,0.273238,0.71,2021,VNM,Vietnam
5717058,1243,57,0.0,0.0,16660,0,0,2258,304,0.273238,0.71,2021,VNM,Vietnam
5717059,1243,57,0.0,0.0,16660,0,0,622,304,0.273238,0.71,2021,VNM,Vietnam
5717060,1243,57,0.0,0.0,16660,0,0,245,304,0.273238,0.71,2021,VNM,Vietnam


In [183]:
todas80['Pais'].unique()

array(['Canada', 'Mexico', 'United States', 'Argentina', 'Brazil',
       'Chile', 'Colombia', 'Ecuador', 'Peru', 'Venezuela',
       'Other South and Central America (BP)', 'Other Caribbean (BP)',
       'Other South America (BP)', 'Austria', 'Belgium', 'Bulgaria',
       'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland',
       'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands',
       'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom', 'Azerbaijan', 'Belarus', 'Kazakhstan', 'Russia',
       'Iran', 'Oman', 'Egypt', 'Morocco', 'South Africa',
       'Eastern Africa (BP)', 'Australia', 'China', 'India', 'Indonesia',
       'Japan', 'New Zealand', 'Pakistan', 'Philippines', 'South Korea',
       'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam'], dtype=object)

In [207]:
ene_ren=pd.read_sql_query('SELECT *FROM "Energia_renovables";', con=conexion)
todas_ren=pd.merge(ene_ren,paises, on='Id_Pais')
todas_ren=pd.merge(todas_ren,años, on='Id_anio')
todas_ren=pd.merge(todas_ren,nuclear, on='Id_anio')
#todas_ren=pd.merge(todas_ren,paises, on='Id_anio')

#todas_ren.drop(columns={'ID_Energia_renovables','Id_Pais_x','Id_anio','Id_Pais_y','ID_Energia_nuclear'},inplace=True)

todas_ren

,ID_Energia_renovables,Id_Pais_x,Id_anio,Consumo_renovables,Produccion_renovables,Codigo_pais,Pais,Anio,ID_Energia_nuclear,Id_Pais_y,Consumo_Nuclear,Produccion_Nuclear
0,2761,44,31,0.082697,0.082697,CAN,Canada,1995,1,44,0.99,97.2
1,2761,44,31,0.082697,0.082697,CAN,Canada,1995,2,160,0.09,8.4
2,2761,44,31,0.082697,0.082697,CAN,Canada,1995,3,297,7.24,708.8
3,2761,44,31,0.082697,0.082697,CAN,Canada,1995,4,11,0.07,7.1
4,2761,44,31,0.082697,0.082697,CAN,Canada,1995,5,35,0.03,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...
178843,5244,210,57,0.025920,0.025920,0,Other Asia Pacific (BP),2021,2480,267,0.00,0.0
178844,5244,210,57,0.025920,0.025920,0,Other Asia Pacific (BP),2021,2481,273,0.25,27.8
178845,5244,210,57,0.025920,0.025920,0,Other Asia Pacific (BP),2021,2482,276,0.00,0.0
178846,5244,210,57,0.025920,0.025920,0,Other Asia Pacific (BP),2021,2483,304,0.00,0.0


In [208]:
todas_ren.drop(columns={'ID_Energia_renovables','Id_Pais_x','Id_anio','Id_Pais_y','ID_Energia_nuclear'},inplace=True)

todas_ren

,Consumo_renovables,Produccion_renovables,Codigo_pais,Pais,Anio,Consumo_Nuclear,Produccion_Nuclear
0,0.082697,0.082697,CAN,Canada,1995,0.99,97.2
1,0.082697,0.082697,CAN,Canada,1995,0.09,8.4
2,0.082697,0.082697,CAN,Canada,1995,7.24,708.8
3,0.082697,0.082697,CAN,Canada,1995,0.07,7.1
4,0.082697,0.082697,CAN,Canada,1995,0.03,2.5
...,...,...,...,...,...,...,...
178843,0.025920,0.025920,0,Other Asia Pacific (BP),2021,0.00,0.0
178844,0.025920,0.025920,0,Other Asia Pacific (BP),2021,0.25,27.8
178845,0.025920,0.025920,0,Other Asia Pacific (BP),2021,0.00,0.0
178846,0.025920,0.025920,0,Other Asia Pacific (BP),2021,0.00,0.0


In [212]:
indexN=todas_ren[todas_ren['Pais']=='Other Europe (BP)'].index
todas_ren.drop(indexN, inplace=True)
indexN=todas_ren[todas_ren['Pais']=='Other CIS (BP)'].index
todas_ren.drop(indexN, inplace=True)
indexN=todas_ren[todas_ren['Pais']=='Other Middle East (BP)'].index
todas_ren.drop(indexN, inplace=True)
indexN=todas_ren[todas_ren['Pais']=='Other Western Africa (BP)'].index
todas_ren.drop(indexN, inplace=True)
indexN=todas_ren[todas_ren['Pais']=='Other Northern Africa (BP)'].index
todas_ren.drop(indexN, inplace=True)
indexN=todas_ren[todas_ren['Pais']=='Other Asia Pacific (BP)'].index
todas_ren.drop(indexN, inplace=True)
todas_ren

,Consumo_renovables,Produccion_renovables,Codigo_pais,Pais,Anio,Consumo_Nuclear,Produccion_Nuclear
0,0.082697,0.082697,CAN,Canada,1995,0.99,97.2
1,0.082697,0.082697,CAN,Canada,1995,0.09,8.4
2,0.082697,0.082697,CAN,Canada,1995,7.24,708.8
3,0.082697,0.082697,CAN,Canada,1995,0.07,7.1
4,0.082697,0.082697,CAN,Canada,1995,0.03,2.5
...,...,...,...,...,...,...,...
178751,0.267104,0.267104,VNM,Vietnam,2021,0.00,0.0
178752,0.267104,0.267104,VNM,Vietnam,2021,0.25,27.8
178753,0.267104,0.267104,VNM,Vietnam,2021,0.00,0.0
178754,0.267104,0.267104,VNM,Vietnam,2021,0.00,0.0


In [213]:
todas_lim_pais=todas_ren.groupby(by=['Pais']).mean()
todas_lim_pais

,Consumo_renovables,Produccion_renovables,Anio,Consumo_Nuclear,Produccion_Nuclear
Pais,,,,,
Argentina,0.046091,0.025226,2008.0,0.276502,28.544686
Australia,0.141623,0.137459,2008.0,0.276502,28.544686
Austria,0.080165,0.067910,2008.0,0.276502,28.544686
Azerbaijan,0.000583,0.000583,2008.0,0.276502,28.544686
Belarus,0.001719,0.001296,2008.0,0.276502,28.544686
...,...,...,...,...,...
Ukraine,0.014640,0.013984,2008.0,0.276502,28.544686
United Kingdom,0.421350,0.396020,2008.0,0.276502,28.544686
United States,2.911940,2.144924,2008.0,0.276502,28.544686


In [227]:
todas_lim_pais['total_consumo'] = (todas_lim_pais['Consumo_renovables']+todas_lim_pais['Consumo_Nuclear'])/2
todas_lim_pais['total_produccion'] = (todas_lim_pais['Produccion_renovables']+todas_lim_pais['Produccion_Nuclear'])/2
todas_lim_pais

,Consumo_renovables,Produccion_renovables,Anio,Consumo_Nuclear,Produccion_Nuclear,total_consumo,total_produccion
Pais,,,,,,,
Argentina,0.046091,0.025226,2008.0,0.276502,28.544686,0.161296,14.284956
Australia,0.141623,0.137459,2008.0,0.276502,28.544686,0.209062,14.341072
Austria,0.080165,0.067910,2008.0,0.276502,28.544686,0.178334,14.306298
Azerbaijan,0.000583,0.000583,2008.0,0.276502,28.544686,0.138542,14.272634
Belarus,0.001719,0.001296,2008.0,0.276502,28.544686,0.139110,14.272991
...,...,...,...,...,...,...,...
Ukraine,0.014640,0.013984,2008.0,0.276502,28.544686,0.145571,14.279335
United Kingdom,0.421350,0.396020,2008.0,0.276502,28.544686,0.348926,14.470353
United States,2.911940,2.144924,2008.0,0.276502,28.544686,1.594221,15.344805


In [214]:
todas_lim_año=todas_ren.groupby(by=['Anio']).mean()
todas_lim_año

,Consumo_renovables,Produccion_renovables,Consumo_Nuclear,Produccion_Nuclear
Anio,,,,
1995,0.032831,0.026062,0.257717,25.241304
1996,0.033374,0.026873,0.267174,26.160870
1997,0.035927,0.029160,0.265217,25.982609
1998,0.037777,0.030878,0.269891,26.432609
1999,0.040058,0.033059,0.280109,27.434783
2000,0.042795,0.035948,0.286630,28.054348
2001,0.044805,0.038188,0.292500,28.845652
2002,0.051182,0.042937,0.295761,29.304348
2003,0.055943,0.046619,0.287826,28.715217


In [225]:
todas_lim_año['total_consumo'] = (todas_lim_año['Consumo_renovables']+todas_lim_año['Consumo_Nuclear'])/2
todas_lim_año

,Consumo_renovables,Produccion_renovables,Consumo_Nuclear,Produccion_Nuclear,total_consumo
Anio,,,,,
1995,0.032831,0.026062,0.257717,25.241304,0.145274
1996,0.033374,0.026873,0.267174,26.160870,0.150274
1997,0.035927,0.029160,0.265217,25.982609,0.150572
1998,0.037777,0.030878,0.269891,26.432609,0.153834
1999,0.040058,0.033059,0.280109,27.434783,0.160084
2000,0.042795,0.035948,0.286630,28.054348,0.164713
2001,0.044805,0.038188,0.292500,28.845652,0.168653
2002,0.051182,0.042937,0.295761,29.304348,0.173471
2003,0.055943,0.046619,0.287826,28.715217,0.171885


In [226]:
todas_lim_año['total_producion'] = (todas_lim_año['Produccion_renovables']+todas_lim_año['Produccion_Nuclear'])/2
todas_lim_año

,Consumo_renovables,Produccion_renovables,Consumo_Nuclear,Produccion_Nuclear,total_consumo,total_producion
Anio,,,,,,
1995,0.032831,0.026062,0.257717,25.241304,0.145274,12.633683
1996,0.033374,0.026873,0.267174,26.160870,0.150274,13.093871
1997,0.035927,0.029160,0.265217,25.982609,0.150572,13.005884
1998,0.037777,0.030878,0.269891,26.432609,0.153834,13.231743
1999,0.040058,0.033059,0.280109,27.434783,0.160084,13.733921
2000,0.042795,0.035948,0.286630,28.054348,0.164713,14.045148
2001,0.044805,0.038188,0.292500,28.845652,0.168653,14.441920
2002,0.051182,0.042937,0.295761,29.304348,0.173471,14.673643
2003,0.055943,0.046619,0.287826,28.715217,0.171885,14.380918


In [240]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=todas_lim_año.index, y=todas_lim_año['total_consumo'],
                    mode='lines',
                    name='Energia Limpia' ))
fig.add_trace(go.Scatter(x=ene_fosil_consumo_total_año.index, y=ene_fosil_consumo_total_año['Total_Consumo'],
                    mode='lines',
                    name='Energia fosil'))
fig.update_layout(title='Consumo Energia fosil vs Energia limpia',
                   xaxis_title='Año',
                   yaxis_title='Consumo(Exajoules)')
#st.plotly(fig, use_container_width=True)
fig.show()

In [239]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=todas_lim_pais.index, y=todas_lim_pais['total_consumo'],
                    mode='lines',
                    name='Energia Limpia' ))
fig.add_trace(go.Scatter(x=ene_fosil_consumo_total_pais.index, y=ene_fosil_consumo_total_pais['Total_Consumo'],
                    mode='lines',
                    name='Energia fosil'))
fig.update_layout(title='Consumo Energia fosil vs Energia limpia por país',
                   xaxis_title='Año',
                   yaxis_title='Consumo(Exajoules)')
#st.plotly(fig, use_container_width=True)
fig.show()

In [257]:
#consulta consumo energias por pais
carbon=pd.read_sql_query('SELECT *FROM "Energia_Carbon";', con=conexion)
carbon.rename(columns={'Id_Anio':'Id_anio'},inplace=True)
petroleo=pd.read_sql_query('SELECT *FROM "Energia_petroleo";', con=conexion)
gas_n=pd.read_sql_query('SELECT *FROM "Gas_Natural";', con=conexion)
eolica=pd.read_sql_query('SELECT *FROM "Energia_Eolica";', con=conexion)
hidroe=pd.read_sql_query('SELECT *FROM "Energia_Hidroelectrica";', con=conexion)
solar=pd.read_sql_query('SELECT *FROM "Energia_solar";', con=conexion)
geoter=pd.read_sql_query('SELECT *FROM "Energia_Geotermica";', con=conexion)
nuclear=pd.read_sql_query('SELECT *FROM "Energia_nuclear";', con=conexion)


In [20]:
def grafica_consumo_fosil(pais):
    carbon=pd.read_sql_query('SELECT *FROM "Energia_Carbon";', con=conexion)
    carbon.rename(columns={'Id_Anio':'Id_anio'},inplace=True)
    petroleo=pd.read_sql_query('SELECT *FROM "Energia_petroleo";', con=conexion)
    gas=pd.read_sql_query('SELECT *FROM "Gas_Natural2";', con=conexion)
    ene_fosil=pd.merge(carbon,petroleo, on='Id_anio')
    ene_fosil=pd.merge(ene_fosil,gas,how='left', on='Id_anio')
    ene_fosil.drop(columns={'Id_Pais_x','ID_Energia_petroleo','Id_Pais_y','Id_Energia_Gas'},inplace=True)
    ene_fosil_consumo=ene_fosil.loc[:,('Id_anio','Id_Pais','Consumo_Carbon','Consumo_Petroleo','Consumo_Gas_Natural')]
    ene_fosil_consumo.fillna(0, inplace=True)
    ene_fosil_consumo['Total_Consumo']=ene_fosil_consumo['Consumo_Carbon']+ene_fosil_consumo['Consumo_Petroleo']+ene_fosil_consumo['Consumo_Gas_Natural']
    ene_fosil_consumo_total=pd.merge(ene_fosil_consumo,paises, on='Id_Pais')
    ene_fosil_consumo_total=pd.merge(ene_fosil_consumo_total,años, on='Id_anio')
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Europe (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other CIS (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Middle East (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Western Africa (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Northern Africa (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Asia Pacific (BP)'].index
    ene_fosil_consumo_total.drop(indexN, inplace=True)
    año90=ene_fosil_consumo_total[ene_fosil_consumo_total.Anio>=1990]
    pais=año90[(año90.Pais==pais)].groupby(by=['Anio']).mean()
   
    fig = go.Figure()
    fig.add_trace(go.Bar(
    x=pais.index,
    y=pais['Consumo_Carbon'],
    name='Carbon',
    marker_color= 'lightslategray'
        ))
    fig.add_trace(go.Bar(
    x=pais.index,
    y=pais['Consumo_Petroleo'],
    name='Petroleo',
    marker_color='black'
        ))
    fig.add_trace(go.Bar(
    x=pais.index,
    y=pais['Consumo_Gas_Natural'],
    name='Gas Natural',
    marker_color='yellowgreen'
        ))
    return fig.show()

In [21]:
grafica=grafica_consumo_fosil('United States')
grafica

In [25]:
#FUNCION PARA GRAFICAR ENERGIAS LIMPIAS
def grafica_consumo_limpio(pais):
    eoil=pd.read_sql_query('SELECT *FROM "Energia_Eolica";', con=conexion)
    solar=pd.read_sql_query('SELECT *FROM "Energia_solar";', con=conexion)
    hidro=pd.read_sql_query('SELECT *FROM "Energia_Hidroelectrica";', con=conexion)
    hidro.rename(columns={'Id_Anio':'Id_anio'}, inplace=True)
    geo=pd.read_sql_query('SELECT *FROM "Energia_Geotermica";', con=conexion)
    geo.rename(columns={'Id_Anio':'Id_anio'}, inplace=True)
    nuclear=pd.read_sql_query('SELECT *FROM "Energia_nuclear_con_ceros";', con=conexion)
    todas=pd.merge(solar,eoil, on='Id_anio')
    todas=pd.merge(todas,hidro, on='Id_anio')
    todas.drop(columns={'Id_Pais_x','Id_Pais_y','Id_Energia_Eolica','Id_Energia_Hidroelectrica'},inplace=True)
    todas=pd.merge(todas, años, on='Id_anio')
    todas90=todas[todas['Anio']>=1990]
    todas90=pd.merge(todas90,paises, on='Id_Pais')
    indexN=todas90[todas90['Pais']=='Other Europe (BP)'].index
    todas90.drop(indexN, inplace=True)
    indexN=todas90[todas90['Pais']=='Other CIS (BP)'].index
    todas90.drop(indexN, inplace=True)
    indexN=todas90[todas90['Pais']=='Other Middle East (BP)'].index
    todas90.drop(indexN, inplace=True)
    indexN=todas90[todas90['Pais']=='Other Western Africa (BP)'].index
    todas90.drop(indexN, inplace=True)
    indexN=todas90[todas90['Pais']=='Other Northern Africa (BP)'].index
    todas90.drop(indexN, inplace=True)
    indexN=todas90[todas90['Pais']=='Other Asia Pacific (BP)'].index
    todas90.drop(indexN, inplace=True)
    todas90.drop(columns={'Id_Energia_solar','Id_anio','Id_Pais'},inplace=True)
    todas90['Consumo_Eolica']=todas90['Consumo_Eolica'].astype(float)
    todas90['Produccion_Eolica']=todas90['Produccion_Eolica'].astype(float)
    todas90['Capacidad_instalada_Eolica']=todas90['Capacidad_instalada_Eolica'].astype(float)
    todas90['Consumo_Hidroelectrica']=todas90['Consumo_Hidroelectrica'].astype(float)
    pais_lim1=todas90[(todas90.Pais==pais)].groupby(by=['Anio']).mean()
    geo_nuc=pd.merge(geo, paises, on='Id_Pais')
    geo_nuc=pd.merge(geo_nuc, años, on='Id_anio')
    geo_nuc=pd.merge(geo_nuc,nuclear, on='Id_anio')
    geo_nuc.drop(columns={'Id_Pais_x','Id_Pais_y','Id_anio','Id_Energia_Geotermica','ID_Energia_nuclear'},inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other Europe (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other CIS (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other Middle East (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other Western Africa (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other Northern Africa (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    indexN=geo_nuc[geo_nuc['Pais']=='Other Asia Pacific (BP)'].index
    geo_nuc.drop(indexN, inplace=True)
    geo_nuc['Consumo_Geotermica']=geo_nuc['Consumo_Geotermica'].astype(float)
    pais_lim2=geo_nuc[(geo_nuc.Pais==pais)].groupby(by=['Anio']).mean()
    fig = go.Figure()
    fig.add_trace(go.Bar(
    x=pais_lim1.index,
    y=pais_lim1['Consumo_solar'],
    name='Solar',
    marker_color= 'yellow'
        ))
    fig.add_trace(go.Bar(
    x=pais_lim1.index,
    y=pais_lim1['Consumo_Eolica'],
    name='Eolica',
    marker_color='crimson'
        ))
    fig.add_trace(go.Bar(
    x=pais_lim1.index,
    y=pais_lim1['Consumo_Hidroelectrica'],
    name='Hidroelectrica',
    marker_color='blue'
        ))
    fig.add_trace(go.Bar(
    x=pais_lim2.index,
    y=pais_lim2['Consumo_Geotermica'],
    name='Geotermica',
    marker_color='goldenrod'
        ))
    fig.add_trace(go.Bar(
    x=pais_lim2.index,
    y=pais_lim2['Consumo_Nuclear'],
    name='Nuclear',
    marker_color='red'
        ))
    return fig.show()


In [26]:
limpias=grafica_consumo_limpio('United States')
limpias

In [37]:
todas90.drop(columns={'Id_Energia_solar','Id_anio','Id_Pais'},inplace=True)



KeyError: "['Id_anio', 'Id_Pais', 'Id_Energia_solar'] not found in axis"

In [40]:
todas90.dtypes

Consumo_solar                 float64
Produccion_solar              float64
Instalada_solar                 int64
Consumo_Eolica                 object
Produccion_Eolica              object
Capacidad_instalada_Eolica     object
Produccion_Hidroelectrica     float64
Consumo_Hidroelectrica         object
Anio                            int64
Codigo_pais                    object
Pais                           object
dtype: object

In [41]:
todas90['Consumo_Eolica']=todas90['Consumo_Eolica'].astype(float)
todas90['Produccion_Eolica']=todas90['Produccion_Eolica'].astype(float)
todas90['Capacidad_instalada_Eolica']=todas90['Capacidad_instalada_Eolica'].astype(float)
todas90

,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,0.00,0.4,0,0.03,2.8,0.0,1.064511,3.15,1990,CAN,Canada
1,0.00,0.4,0,0.01,0.6,0.0,1.064511,3.15,1990,CAN,Canada
2,0.00,0.4,0,0.00,0.1,0.0,1.064511,3.15,1990,CAN,Canada
3,0.00,0.4,0,0.00,0.1,0.0,1.064511,3.15,1990,CAN,Canada
4,0.01,0.5,0,0.03,3.0,0.0,1.106271,3.27,1991,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...
5716915,0.00,0.0,16660,0.00,0.0,1514.0,0.273238,0.71,2021,VNM,Vietnam
5716916,0.00,0.0,16660,0.00,0.0,2258.0,0.273238,0.71,2021,VNM,Vietnam
5716917,0.00,0.0,16660,0.00,0.0,622.0,0.273238,0.71,2021,VNM,Vietnam
5716918,0.00,0.0,16660,0.00,0.0,245.0,0.273238,0.71,2021,VNM,Vietnam


In [43]:
todas90['Consumo_Hidroelectrica']=todas90['Consumo_Hidroelectrica'].astype(float)
todas90

,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Produccion_Hidroelectrica,Consumo_Hidroelectrica,Anio,Codigo_pais,Pais
0,0.00,0.4,0,0.03,2.8,0.0,1.064511,3.15,1990,CAN,Canada
1,0.00,0.4,0,0.01,0.6,0.0,1.064511,3.15,1990,CAN,Canada
2,0.00,0.4,0,0.00,0.1,0.0,1.064511,3.15,1990,CAN,Canada
3,0.00,0.4,0,0.00,0.1,0.0,1.064511,3.15,1990,CAN,Canada
4,0.01,0.5,0,0.03,3.0,0.0,1.106271,3.27,1991,CAN,Canada
...,...,...,...,...,...,...,...,...,...,...,...
5716915,0.00,0.0,16660,0.00,0.0,1514.0,0.273238,0.71,2021,VNM,Vietnam
5716916,0.00,0.0,16660,0.00,0.0,2258.0,0.273238,0.71,2021,VNM,Vietnam
5716917,0.00,0.0,16660,0.00,0.0,622.0,0.273238,0.71,2021,VNM,Vietnam
5716918,0.00,0.0,16660,0.00,0.0,245.0,0.273238,0.71,2021,VNM,Vietnam


In [44]:
pais_lim1=todas90[(todas90.Pais=='Canada')].groupby(by=['Anio']).mean()
pais_lim1

,Consumo_solar,Produccion_solar,Instalada_solar,Consumo_Eolica,Produccion_Eolica,Capacidad_instalada_Eolica,Produccion_Hidroelectrica,Consumo_Hidroelectrica
Anio,,,,,,,,
1990,0.000000,0.400000,0.000000,0.010000,0.900000,0.000000,1.064511,3.15
1991,0.010000,0.500000,0.000000,0.008000,0.800000,0.000000,1.106271,3.27
1992,0.000000,0.400000,0.000000,0.005000,0.575000,0.000000,1.134711,3.36
1993,0.010000,0.500000,0.000000,0.005000,0.560000,0.000000,1.159911,3.43
1994,0.010000,0.500000,0.000000,0.006000,0.700000,0.000000,1.180431,3.49
1995,0.010000,0.500000,0.000000,0.008000,0.820000,0.000000,1.202750,3.56
1996,0.000625,0.037500,10.750000,0.008000,0.900000,0.000000,1.276550,3.77
1997,0.000588,0.035294,13.588235,0.003611,0.330556,211.750000,1.255310,3.71
1998,0.000526,0.031579,16.000000,0.003947,0.418421,261.473684,1.191230,3.52


In [35]:
geo_nuc=pd.merge(geo, paises, on='Id_Pais')
geo_nuc=pd.merge(geo_nuc, años, on='Id_anio')
geo_nuc=pd.merge(geo_nuc,nuclear, on='Id_anio')
geo_nuc.drop(columns={'Id_Pais_x','Id_Pais_y','Id_anio','Id_Energia_Geotermica','ID_Energia_nuclear'},inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other Europe (BP)'].index
geo_nuc.drop(indexN, inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other CIS (BP)'].index
geo_nuc.drop(indexN, inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other Middle East (BP)'].index
geo_nuc.drop(indexN, inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other Western Africa (BP)'].index
geo_nuc.drop(indexN, inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other Northern Africa (BP)'].index
geo_nuc.drop(indexN, inplace=True)
indexN=geo_nuc[geo_nuc['Pais']=='Other Asia Pacific (BP)'].index
geo_nuc.drop(indexN, inplace=True)

geo_nuc

,Produccion_Geotermica,Consumo_Geotermica,Codigo_pais,Pais,Anio,Consumo_Nuclear,Produccion_Nuclear
0,0.02556,0.08,CAN,Canada,1995,0.99,97.2
1,0.02556,0.08,CAN,Canada,1995,0.09,8.4
2,0.02556,0.08,CAN,Canada,1995,7.24,708.8
3,0.02556,0.08,CAN,Canada,1995,0.07,7.1
4,0.02556,0.08,CAN,Canada,1995,0.03,2.5
...,...,...,...,...,...,...,...
176267,0.00072,0,VNM,Vietnam,2021,0.00,0.0
176268,0.00072,0,VNM,Vietnam,2021,0.25,27.8
176269,0.00072,0,VNM,Vietnam,2021,0.00,0.0
176270,0.00072,0,VNM,Vietnam,2021,0.00,0.0


In [45]:
geo_nuc.dtypes

Produccion_Geotermica    float64
Consumo_Geotermica        object
Codigo_pais               object
Pais                      object
Anio                       int64
Consumo_Nuclear          float64
Produccion_Nuclear       float64
dtype: object

In [46]:
geo_nuc['Consumo_Geotermica']=geo_nuc['Consumo_Geotermica'].astype(float)

In [47]:
pais_lim2=geo_nuc[(geo_nuc.Pais=='Canada')].groupby(by=['Anio']).mean()
pais_lim2

,Produccion_Geotermica,Consumo_Geotermica,Consumo_Nuclear,Produccion_Nuclear
Anio,,,,
1995,0.02556,0.08,0.257717,25.241304
1996,0.02700,0.09,0.267174,26.160870
1997,0.02952,0.09,0.265217,25.982609
1998,0.03240,0.10,0.269891,26.432609
1999,0.03204,0.10,0.280109,27.434783
2000,0.03204,0.10,0.286630,28.054348
2001,0.03492,0.11,0.292500,28.845652
2002,0.03600,0.12,0.295761,29.304348
2003,0.03384,0.11,0.287826,28.715217


In [48]:
fig = go.Figure()
fig.add_trace(go.Bar(
x=pais_lim1.index,
y=pais_lim1['Consumo_solar'],
name='Solar',
marker_color= 'lightslategray'
    ))
fig.add_trace(go.Bar(
x=pais_lim1.index,
y=pais_lim1['Consumo_Eolica'],
name='Eolica',
marker_color='crimson'
    ))
fig.add_trace(go.Bar(
x=pais_lim1.index,
y=pais_lim1['Consumo_Hidroelectrica'],
name='Hidroelectrica',
marker_color='indianred'
    ))
fig.add_trace(go.Bar(
x=pais_lim2.index,
y=pais_lim2['Consumo_Geotermica'],
name='Geotermica',
marker_color='lightsalmon'
    ))
fig.add_trace(go.Bar(
x=pais_lim2.index,
y=pais_lim2['Consumo_Nuclear'],
name='Nuclear',
marker_color='indianred'
    ))
fig.show()